## Figure 5: Climate baseline

In [ ]:
# spatial libraries
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe

import pandas as pd
import numpy as np
import cftime
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

os.chdir('/home/rooda/Dropbox/Patagonia/')
period = slice("1980-01-01", "2019-12-31")

In [ ]:
# Basins
basins = gpd.read_file("GIS South/Basins_Patagonia_all.shp")
basins = basins.set_index("ID")

In [ ]:
# Climate baseline (1980-2019)
pp_pmet   = xr.open_dataset("Data/Precipitation/PP_PMET_1980_2020m.nc").sel(time = period).rename({'latitude':'lat', 'longitude':'lon'}).drop("crs")
pp_era5   = xr.open_dataset("Data/Precipitation/PP_ERA5_1959_2021m.nc").sel(time = period)
pp_cr2met = xr.open_dataset("Data/Precipitation/PP_CR2MET_1979_2020m.nc").sel(time = period)

t2m_pmet   = xr.open_dataset("Data/Temperature/Tavg_PMET_1980_2020m.nc").sel(time = period).rename({'latitude':'lat', 'longitude':'lon'})
t2m_era5   = xr.open_dataset("Data/Temperature/Tavg_ERA5_1959_2021m.nc").sel(time = period)
t2m_cr2met = xr.open_dataset("Data/Temperature/Tavg_CR2MET_1979_2020m.nc").sel(time = period)

In [ ]:
# Average annual value for selected period
pp_pmet    = pp_pmet.resample(time='1Y').sum().mean(dim="time")
pp_era5    = pp_era5.resample(time='1Y').sum().mean(dim="time")
pp_cr2met  = pp_cr2met.resample(time='1Y').sum().mean(dim="time")

t2m_pmet   = t2m_pmet.resample(time='1Y').mean().mean(dim="time")
t2m_era5   = t2m_era5.resample(time='1Y').mean().mean(dim="time")
t2m_cr2met = t2m_cr2met.resample(time='1Y').mean().mean(dim="time")

In [ ]:
# Correction of zero values
pp_pmet = pp_pmet.where(pp_pmet > 0)
pp_cr2met = pp_cr2met.where(pp_cr2met > 0)

In [ ]:
pp_pmet_avg    = xe.SpatialAverager(pp_pmet,   basins.geometry, geom_dim_name="avg")
pp_era5_avg    = xe.SpatialAverager(pp_era5,   basins.geometry, geom_dim_name="avg")
pp_cr2met_avg  = xe.SpatialAverager(pp_cr2met, basins.geometry, geom_dim_name="avg")

t2m_pmet_avg   = xe.SpatialAverager(t2m_pmet,   basins.geometry, geom_dim_name="avg")
t2m_era5_avg   = xe.SpatialAverager(t2m_era5,   basins.geometry, geom_dim_name="avg")
t2m_cr2met_avg = xe.SpatialAverager(t2m_cr2met, basins.geometry, geom_dim_name="avg")

In [ ]:
basins["PP_PMET"]   = pp_pmet_avg(pp_pmet.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ERA5"]   = pp_era5_avg(pp_era5.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_CR2MET"] = pp_cr2met_avg(pp_cr2met.pp, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ERA5d"]  = basins.PP_ERA5 - basins.PP_PMET
basins["PP_CR2METd"]= basins.PP_CR2MET - basins.PP_PMET

basins["T2M_PMET"]   = t2m_pmet_avg(t2m_pmet.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ERA5"]   = t2m_era5_avg(t2m_era5.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_CR2MET"] = t2m_cr2met_avg(t2m_cr2met.t2m, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ERA5d"]  =  basins.T2M_ERA5 - basins.T2M_PMET
basins["T2M_CR2METd"]=  basins.T2M_CR2MET - basins.T2M_PMET

In [ ]:
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

from shapely.geometry import Polygon
poly_gdf = Polygon([(-76, -55.7), (-76, -40.6), (-68.1, -40.6), (-68.1, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
fig = make_subplots(rows=2, cols=3, column_titles = ["PMET", "ERA5", "CR2MET"], horizontal_spacing = 0.01, vertical_spacing = 0.01, 
                    specs=[[{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}],
                           [{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}]])

# Precipitation -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_PMET'], 
                            colorscale = ["#ccebc5", "#4eb3d3", "#034b8a"], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 0, zmax = 8000, colorbar=dict(len=0.25, x=0.23, y= 0.87, title='PP (mm)', thickness=20)), row=1, col=1)

fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=2)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['PP_ERA5d'], 
                            colorscale="Earth", marker_line_color='white', marker_line_width=0.1, 
                            colorbar=dict(len=0.25, x=0.89, y = 0.87, title='ΔPP (mm)', thickness=20), zmid = 0, zmax = 2000, zmin = -2000), row=1, col=2)

fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=3)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['PP_CR2METd'],
                            colorscale="Earth", marker_line_color='white', marker_line_width=0.1, 
                            showscale= False, zmid = 0, zmax = 2000, zmin = -2000), row=1, col=3)

# Temperature -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=1)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_PMET'], 
                            colorscale="Sunset_r", marker_line_color='white', marker_line_width=0.1, 
                            zmin = -2, zmax = 8, colorbar=dict(len=0.25, x=0.23, y= 0.37, title='T2M (ºC)', thickness=20)), row=2, col=1)

fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=2)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_ERA5d'], 
                            colorscale="Earth_r", marker_line_color='white', marker_line_width=0.1, 
                            colorbar=dict(len=0.25, x=0.89, y = 0.37, title='ΔT2M (ºC)', thickness=20), zmid = 0, zmax = 2, zmin = -2), row=2, col=2)

fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=3)

fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_CR2METd'],
                            colorscale="Earth_r",  marker_line_color='white', marker_line_width=0.1, 
                            showscale= False, zmid = 0, zmax = 2, zmin = -2), row=2, col=3)

fig.update_layout(autosize = False, width = 900, height = 1150, margin = dict(l=10, r=5, b=5, t=30, pad=0, autoexpand=True))
fig.update_geos(showframe = True, framewidth = 1, lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)
fig.show()

In [ ]:
cl     = px.colors.qualitative.D3

fig = make_subplots(rows=2, cols=3, shared_yaxes = True, shared_xaxes = False, horizontal_spacing = 0.02, vertical_spacing = 0.05, 
                    subplot_titles = ("PMET", "CR2MET", "ERA5"), y_title = ("Annual Temperature (°C)                              Annual Precipitation (mm)"))

fig.add_trace(go.Histogram(y = pp_pmet_np,   marker_color = cl[0], histnorm='percent', ybins=dict(start=0, size=200, end=8000)), row=1, col=1)
fig.add_trace(go.Histogram(y = pp_cr2met_np, marker_color = cl[1], histnorm='percent', ybins=dict(start=0, size=200, end=8000)), row=1, col=2)
fig.add_trace(go.Histogram(y = pp_era5_np,   marker_color = cl[2], histnorm='percent', ybins=dict(start=0, size=200, end=8000)), row=1, col=3)
fig.update_yaxes(dtick = 2000, row=1, col=1)
fig.update_yaxes(dtick = 2000, row=1, col=2)
fig.update_yaxes(dtick = 2000, row=1, col=3)

fig.add_trace(go.Histogram(y = t2m_pmet_np,  marker_color= cl[0], histnorm='percent', ybins=dict(start=-6, size=0.3, end=9)), row=2, col=1)
fig.add_trace(go.Histogram(y = t2m_cr2met_np,marker_color= cl[1], histnorm='percent', ybins=dict(start=-6, size=0.3, end=9)), row=2, col=2)
fig.add_trace(go.Histogram(y = t2m_era5_np,  marker_color= cl[2], histnorm='percent', ybins=dict(start=-6, size=0.3, end=9)), row=2, col=3)
fig.update_yaxes(dtick = 3, row=2, col=1)
fig.update_yaxes(dtick = 3, row=2, col=2)
fig.update_yaxes(dtick = 3, row=2, col=3)

fig.update_layout(height=800, width=700, template = "seaborn", showlegend = False, autosize=False,)
#fig.update_layout(margin = {'l':0.2,'r':0.2,'t':0.2,'b':0.2})
fig.update_xaxes(ticks="outside", dtick = 2)
fig.update_yaxes(ticks="outside")
fig.update_traces(opacity=0.7)
fig.show()


In [ ]:
os.chdir("/home/rooda/OGGM_results/Future_climate/")

lat_coords = np.arange(-56,-40, 0.5)
lon_coords = np.arange(-75,-67, 0.5)

baseline_period = slice("1980-01-01", "2010-01-01")
future_period   = slice("2070-01-01", "2100-01-01")

gcm_list  = ["ACCESS-CM2", "CMCC-ESM2", "EC-Earth3", "GFDL-ESM4", "INM-CM5-0", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6", "NorESM2-MM"]
ssp_list  = ['ssp126', 'ssp245', 'ssp585']

results_pp = []
results_t2m = []

for ssp in ssp_list:
    
    results_gcm_pp  = []
    results_gcm_t2m = []
    
    for gcm in gcm_list:
        
        pp_model_ssp = xr.open_dataset("PP_" + gcm + "_" + ssp + ".nc")["pr"]
        pp_model_ssp = pp_model_ssp.interp(lat = lat_coords, lon = lon_coords)
        pp_model_ssp = pp_model_ssp.resample(time='YS').sum()
        pp_change    = (pp_model_ssp.sel(time = future_period).mean(dim="time") / pp_model_ssp.sel(time = baseline_period).mean(dim="time"))-1
        results_gcm_pp.append(pp_change)
        
        t2m_model_ssp = xr.open_mfdataset("T2M_" + gcm + "_" + ssp + ".nc")["tas"]
        t2m_model_ssp = t2m_model_ssp.interp(lat = lat_coords, lon = lon_coords)
        t2m_model_ssp = t2m_model_ssp.resample(time='YS').mean()
        t2m_change    = t2m_model_ssp.sel(time = future_period).mean(dim="time") - t2m_model_ssp.sel(time = baseline_period).mean(dim="time")
        results_gcm_t2m.append(t2m_change)
        print(ssp, gcm)
        
    results_gcm_pp  = xr.concat(results_gcm_pp,  dim='gcm').mean(dim = "gcm")
    results_gcm_t2m = xr.concat(results_gcm_t2m, dim='gcm').mean(dim = "gcm")
    results_pp.append(results_gcm_pp)
    results_t2m.append(results_gcm_t2m)
    
final_pp  = xr.concat(results_pp,  dim='ssp')
final_t2m = xr.concat(results_t2m, dim='ssp')

In [ ]:
final_t2m[0].plot()

In [ ]:
fig  = px.imshow(final_t2m[0], color_continuous_scale='RdBu_r', origin='lower')
fig.show()

In [ ]:
?px.imshow